# Ledgera (iOS) — Dual-Approach Banking + Projections + Line-by-Line Accounting

## 0. Summary

Ledgera is an iOS app that combines:

1. **Projection / planning** (income → tax → living expenses → savings allocation → account growth), and
2. **Granular accounting** (minute-level, line-by-line ledger with auditability).

Design principle: **Ledger Core is the single source of truth (actuals)**; the **Planning Layer** consumes ledger data and user assumptions to generate forecasts and targets.

---

## 1. Core Product Requirements

### 1.1 Projection Capability (Spreadsheet-grade)

* Multi-currency inputs/outputs (e.g., SGD base with USD/AED/INR views).
* Monthly salary + bonus modelling (frequency, size, and seasonality).
* Taxes (MVP: flat rate; later: bracketed regimes).
* Living expenses broken down by category with inflation assumptions.
* Savings and fund allocation weights that sum to 100%.
* Roll-forward by month and year producing tables similar to the user’s sheet:

  * Income after tax
  * Living expenses
  * Investable income
  * Cash reserve growth
  * Investment growth
  * Wealth trajectory and target tracking

### 1.2 Minute-Level Accounting

* Transaction journal view with filters/search.
* Support for:

  * Category assignment
  * Splits
  * Transfers
  * Refunds
  * Attachments (receipts)
  * Reconciliation state
* Correctness via **double-entry accounting** internally.

### 1.3 “Dual Approach Banking Setup”

* User defines a banking stack (cash, semi-liquid, emergency, investments, buckets).
* Wizard proposes:

  * account structure
  * target buffers (e.g., 6 months)
  * recurring transfers based on plan
* Variance tracking: planned vs actual spending/savings.

---

## 2. UX / Screens

### 2.1 Home Dashboard

* Net worth (base currency + optional per-currency)
* Cash runway (months)
* Current month: income, spend, savings rate
* “On track / Off track” vs plan

### 2.2 Accounts

* Account list and balances
* Drill-down transaction feed
* Reconciliation (v1)

### 2.3 Ledger (Accounting)

* Journal: chronological list, search, filters (account/category/tag/date)
* Add transaction:

  * Quick entry
  * Advanced: split lines, transfers, FX legs

### 2.4 Plan (Projection)

* Assumptions editor:

  * income streams
  * tax
  * expense categories with inflation
  * allocation weights
  * expected return rates per bucket
* Outputs:

  * annual summary
  * month-by-month roll-forward
  * scenario comparison (v1)

### 2.5 Targets

* Targets by bucket (e.g., “AED 100k by Dec 2026”)
* Required monthly contribution, progress vs plan

---

## 3. Accounting Model (Double-Entry)

### 3.1 Entities

* **Account**: asset / liability / income / expense / equity
* **Transaction**: metadata (timestamp, payee, memo, source)
* **Posting**: individual debit/credit lines, amounts, currency
* **Commodity**: currency (later: securities)
* **Price**: FX rates and (later) security prices

### 3.2 Invariants

Each transaction must balance:

* Sum of postings in native commodities balances per commodity OR
* Store base-currency equivalents per posting using an FX rate at booking time and enforce base-sum == 0.

### 3.3 Multi-currency Patterns

* Spend in AED from AED account:

  * Debit Groceries Expense (AED)
  * Credit AED Checking (AED)
  * Store FX rate used for base reporting

* FX transfer AED → SGD:

  * Debit SGD Checking (SGD)
  * Credit AED Checking (AED)
  * Optional: FX gain/loss account (strict accounting)

---

## 4. Projection Engine

### 4.1 Deterministic Baseline (MVP)

Per month t:

* Income after tax: I_t = (salary_t + bonus_t + other_t) * (1 - tax_rate_t)
* Expenses with inflation: E_t = E_0 * (1 + pi)^(t/12)
* Savings: S_t = I_t - E_t
* Allocation by bucket k: A_{t,k} = S_t * w_k
* Roll-forward: B_{t+1,k} = B_{t,k} * (1 + r_k)^(1/12) + A_{t,k}

### 4.2 Stochastic Upgrade (later)

* Monte Carlo over investment returns
* Probability of hitting targets
* Downside risk paths

---

## 5. iOS Architecture

### 5.1 Stack

* SwiftUI UI layer
* Persistence: SQLite (recommended) via GRDB OR Core Data
* Sync: optional (CloudKit first, custom backend later)
* Security: on-device encryption, Keychain

### 5.2 Layers

* Domain: accounting and projection logic
* Data: repositories, migration handling
* Services:

  * import
  * categorization rules
  * projection engine
* Presentation: view models and derived metrics

### 5.3 Performance

* Index by timestamp, account_id
* Precompute aggregates for dashboards
* Cache balance checkpoints

---

## 6. Minimal Database Schema (Draft)

* accounts(id, name, type, currency, institution, created_at)
* transactions(id, timestamp, payee, memo, status, source)
* postings(id, transaction_id, account_id, amount, currency, base_amount, fx_rate)
* categories(id, name, parent_id)
* tags(id, name)
* transaction_tags(transaction_id, tag_id)
* prices(id, timestamp, base_ccy, quote_ccy, rate, source)
* scenarios(id, name)
* projection_assumptions(id, scenario_id, key, value, currency)
* projection_results(id, scenario_id, period, metric_key, value, currency)

---

## 7. MVP Scope

1. Accounts + transaction journal
2. Categories + splits + transfers
3. Multi-currency with manual FX table
4. Projection: income/tax/expenses/allocations/returns
5. Targets: required monthly contribution and tracking

---

## 8. V1 / V2 Add-ons

* Rules-based auto-categorization
* CSV import and mapping
* Reconciliation
* Scenario comparison
* Investment tracking (NAV series, later lots)
* Monte Carlo goal probability
* Tax regimes (brackets)
* Shared/family vault

---

## 9. Open Decisions

* Base currency selection: fixed vs per-profile
* FX rate source policy (manual vs market feed)
* Strict accounting for FX gains/loss vs simplified reporting
* Sync: local-only mode vs cloud-first

---

## 10. Next Deliverable

Convert this into:

* detailed screen-by-screen requirements
* API/DTOs for domain layer
* migration plan
* test fixtures
* backlog (epics/stories) for build execution


# Ledgera (iOS) — Dual-Approach Banking + Projections + Line-by-Line Accounting

## 0. Summary

Ledgera is an iOS app that combines:

1. **Projection / planning** (income → tax → living expenses → savings allocation → account growth), and
2. **Granular accounting** (minute-level, line-by-line ledger with auditability).

Design principle: **Ledger Core is the single source of truth (actuals)**; the **Planning Layer** consumes ledger data and user assumptions to generate forecasts and targets.

---

## 1. Core Product Requirements

### 1.1 Projection Capability (Spreadsheet-grade)

**Goal:** replicate and exceed the sheet workflow: multi-currency income → tax → living expenses → savings allocation → bucket growth → target tracking.

**Inputs**

* **Profile settings**

  * Base currency (user-selectable; default SGD)
  * Display currencies (USD/AED/INR etc.)
  * Timezone and fiscal year start
* **FX rates**

  * Manual rates (override)
  * Market rates via Price Service (Yahoo Finance initially)
  * Choice of valuation policy: end-of-day vs booking-time vs “latest” (MVP default: latest for dashboards; booking-time snapshot for transaction reporting)
* **Income streams**

  * Salary: amount + pay frequency (monthly/biweekly) + currency
  * Bonus: schedule (annual/quarterly), amount or % of base, currency
  * Other: irregular income with optional recurrence
* **Taxes**

  * MVP: flat % rate per income stream or global flat rate
  * Later: bracketed regimes and residency toggles
* **Expenses**

  * Fixed monthly items by category (rent, utilities, internet, phone)
  * Variable budget envelopes by category (groceries, dining, transport, health, work)
  * Inflation model: per-category inflation (MVP: global inflation + per-category override)
* **Allocations / buckets**

  * Buckets: Cash Reserve (semi-liquid), Emergency, Investments, Travel, Giving/Lifestyle
  * Allocation weights w_k sum to 100%
  * Constraints: minimum cash buffer policy (e.g., 6 months) and target amounts
* **Returns**

  * Per-bucket expected return r_k (MVP deterministic)
  * Later: stochastic return models

**Outputs**

* Monthly and annual roll-forward tables
* Savings rate, variance vs plan
* Target progress: probability later, deterministic trajectory now

### 1.2 Minute-Level Accounting (Ledger Core)

**Goal:** a trustworthy, auditable ledger at line-item granularity.

**Must support**

* Manual transaction entry (fast path + advanced split)
* CSV import (bank statements / exported transactions)
* Categorization + tags + notes + attachments
* Transfers across accounts (including cross-currency)
* Reconciliation states: Unreconciled / Reconciled / Pending
* Balance computation by account at any timestamp

**Correctness**

* Double-entry storage internally (even if UI looks single-entry)
* Immutable journal entry IDs; edits create revisions (MVP: soft edits with audit fields; V1: explicit revision chain)

### 1.3 Dual Approach Banking Setup

**Goal:** the app recommends and operationalizes a “banking stack” aligned to the plan.

**Wizard**

* Choose base + display currencies
* Create accounts (checking, semi-liquid, emergency, broker, credit cards)
* Set buffer policy (e.g., 6 months living expenses)
* Set allocation weights + targets

**Automation (non-bank-integrated)**

* Generate “suggested transfers” and reminders
* Surface drift: “cash reserve below band, move S$X”

---

## 2. UX / Screens

### 2.1 Home Dashboard

* Net worth (base currency + optional per-currency)
* Cash runway (months)
* Current month: income, spend, savings rate
* “On track / Off track” vs plan

### 2.2 Accounts

* Account list and balances
* Drill-down transaction feed
* Reconciliation (v1)

### 2.3 Ledger (Accounting)

* Journal: chronological list, search, filters (account/category/tag/date)
* Add transaction:

  * Quick entry
  * Advanced: split lines, transfers, FX legs

### 2.4 Plan (Projection)

* Assumptions editor:

  * income streams
  * tax
  * expense categories with inflation
  * allocation weights
  * expected return rates per bucket
* Outputs:

  * annual summary
  * month-by-month roll-forward
  * scenario comparison (v1)

### 2.5 Targets

* Targets by bucket (e.g., “AED 100k by Dec 2026”)
* Required monthly contribution, progress vs plan

---

## 3. Accounting Model (Double-Entry)

### 3.1 Entities

* **Account**: asset / liability / income / expense / equity
* **Transaction**: metadata (timestamp, payee, memo, source)
* **Posting**: individual debit/credit lines, amounts, currency
* **Commodity**: currency (later: securities)
* **Price**: FX rates and (later) security prices

### 3.2 Invariants

Each transaction must balance:

* Sum of postings in native commodities balances per commodity OR
* Store base-currency equivalents per posting using an FX rate at booking time and enforce base-sum == 0.

### 3.3 Multi-currency Patterns

* Spend in AED from AED account:

  * Debit Groceries Expense (AED)
  * Credit AED Checking (AED)
  * Store FX rate used for base reporting

* FX transfer AED → SGD:

  * Debit SGD Checking (SGD)
  * Credit AED Checking (AED)
  * Optional: FX gain/loss account (strict accounting)

---

## 4. Projection Engine

### 4.1 Deterministic Baseline (MVP)

Define months t = 0..T.

**Income after tax**

* Aggregate income streams by month and currency.
* Convert to base currency using PriceService or manual overrides.
* Apply tax (flat):

  * I_t = (salary_t + bonus_t + other_t) * (1 - tax_rate_t)

**Expenses with inflation**

* Expenses are category-based. Each category c has base monthly spend E_{0,c} and inflation pi_c.
* E_{t,c} = E_{0,c} * (1 + pi_c)^(t/12)
* Total expenses: E_t = sum_c E_{t,c}

**Savings**

* S_t = I_t - E_t

**Allocation**

* Buckets k with weights w_k, sum_k w_k = 1
* A_{t,k} = max(0, S_t) * w_k
* Optional: if S_t < 0, drawdown policy from cash reserve first (V1)

**Bucket roll-forward**

* B_{t+1,k} = B_{t,k} * (1 + r_k)^(1/12) + A_{t,k}

**Targets**

* For each target (bucket k, amount X, date T*):

  * Track projected B_{t,k} vs X
  * Compute required additional monthly contribution (simple):

    * delta = max(0, X - B_{T*,k})
    * req = delta / months_remaining

### 4.2 Market Data Integration (MVP)

**Goal:** keep accounting simplified but still provide live valuation and realistic projections.

* **FX**

  * Use Yahoo Finance symbols (e.g., SGDUSD=X, USDAED=X) or equivalent mapping.
  * Cache rates with TTL (e.g., 6h) and store locally.
* **Investments**

  * For investment buckets, allow either:

    1. manual expected return r_k, or
    2. map a bucket to a proxy ticker (e.g., global ETF) and compute trailing return estimates (V1)

### 4.3 Stochastic Upgrade (later)

* Monte Carlo over investment returns with correlation structure
* Probability of hitting targets, downside percentiles

---

## 5. iOS Architecture

### 5.1 Stack (Build-Start Setup)

* **UI:** SwiftUI
* **Persistence:** SQLite via **GRDB** (preferred for explicit migrations + performance)
* **Sync:** **CloudKit day 1**

  * CloudKit as system of record for synced users
  * Local SQLite remains the fast local cache
  * Conflict handling: last-write-wins for metadata; ledger postings are append-only where possible
* **Security:**

  * iOS Data Protection (file-level)
  * Keychain for secrets (if any)
  * Optional local-only mode can be added later; initial build assumes iCloud enabled

### 5.2 Modules

* **Domain (pure Swift)**

  * Ledger: Accounts, Transactions, Postings, Balancing
  * Reporting: balances, cashflow summaries
  * Planning: scenario inputs, deterministic projection engine
* **Data layer**

  * GRDB repositories
  * CloudKit sync adapter
  * Import pipelines (CSV → normalized transactions)
* **Services**

  * **PriceService** (Yahoo Finance initially)

    * FX quotes (e.g., SGDUSD=X style)
    * ETFs/stocks for investment bucket valuation
    * Caching + rate limiting
  * Categorization rules engine (V1)

### 5.3 CloudKit Data Strategy (Day 1)

**Principles**

* Store canonical entities in CloudKit records: Account, Transaction, Posting, Category, Tag, PriceSnapshot (optional)
* Use stable UUIDs as primary keys across local/remote
* Prefer **append-only** for ledger-critical entities:

  * Transactions are immutable; corrections create adjusting transactions
  * If MVP supports edits, maintain `updated_at` + `original_id` references

**Sync mechanics**

* Background sync on app open + periodic refresh
* Push subscriptions for updates
* Local GRDB tables include `ck_record_name`, `ck_change_tag`, `sync_state`

### 5.4 Performance

* Index: timestamp, account_id, category_id
* Maintain daily/monthly aggregates tables for dashboards
* Balance checkpoints by account (e.g., monthly) to avoid O(N) recompute

---

## 6. Minimal Database Schema (Draft)

* accounts(id, name, type, currency, institution, created_at)
* transactions(id, timestamp, payee, memo, status, source)
* postings(id, transaction_id, account_id, amount, currency, base_amount, fx_rate)
* categories(id, name, parent_id)
* tags(id, name)
* transaction_tags(transaction_id, tag_id)
* prices(id, timestamp, base_ccy, quote_ccy, rate, source)
* scenarios(id, name)
* projection_assumptions(id, scenario_id, key, value, currency)
* projection_results(id, scenario_id, period, metric_key, value, currency)

---

## 7. MVP Scope

1. Accounts + transaction journal
2. Categories + splits + transfers
3. Multi-currency with manual FX table
4. Projection: income/tax/expenses/allocations/returns
5. Targets: required monthly contribution and tracking

---

## 8. V1 / V2 Add-ons

* Rules-based auto-categorization
* CSV import and mapping
* Reconciliation
* Scenario comparison
* Investment tracking (NAV series, later lots)
* Monte Carlo goal probability
* Tax regimes (brackets)
* Shared/family vault

---

## 9. Decisions (Locked for Build v0)

1. **Base currency:** user-selectable per profile (default: SGD).
2. **Sync:** CloudKit from day 1 (with local caching).
3. **Data ingest:** CSV import + manual entry (no direct bank aggregation initially).
4. **Market data:** simplified P&L accounting, but **API-based prices** for FX / securities using a lightweight provider (initially Yahoo Finance / yfinance-compatible endpoints).

---

## 10. Next Deliverable

Convert this into:

* detailed screen-by-screen requirements
* API/DTOs for domain layer
* migration plan
* test fixtures
* backlog (epics/stories) for build execution
